In [1]:
!pip install pandas
!pip install openpyxl

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [164]:
import pandas as pd
import re
import sqlite3
def read_derived_rules(column_name):
    query = f"""
    SELECT {column_name} FROM T3_table
    WHERE Type = 'Derive' AND [OD rules] IS NOT NULL AND [OD rules] LIKE '%✔%'
    """
    conn = sqlite3.connect('T3.db')
    cursor = conn.cursor()
    cursor.execute(query)
    data = cursor.fetchall()
    conn.close()
    return [row[0] for row in data if row[0] is not None]
components = read_derived_rules('Component')
project_name = read_derived_rules('[Project name]')
version = read_derived_rules('Version')
date = read_derived_rules('Date')
owner = read_derived_rules('Owner')
OD_rules_raw = read_derived_rules('[OD rules]')
OD_rules = [re.sub(r'\bOnly\b.*', '', rule).strip() for rule in OD_rules_raw]
OD_rules = [re.sub(r'\bSBB\w*\b', '', rule).strip() for rule in OD_rules]
OD_rules = [re.sub(r'[()]', '', rule).strip() for rule in OD_rules]
rules = read_derived_rules('rules')
newrules = [line.split(']')[0] for line in rules]
pattern = r'([✔✘])\s*(.*)'
data = []
section_id = 1
for section in OD_rules:
    items = re.split(r'\s*\|\|\s*|\s*&&\s*|\s*\(\s*|\s*\)\s*', section)
    section_data = []
    for item in items:
        match = re.match(pattern, item)
        if match:
            status, name = match.groups()
            section_data.append((section_id, status, name))
    if section_data:
        data.extend(section_data)
    section_id += 1
df = pd.DataFrame(data, columns=['Section_id', 'Status', 'name'])
print(df)

       Section_id Status                                               name
0               1      ✔      Intel i3-12100 3.3GHz/4C/8T/12M 60W DDR5 4800
1               2      ✔  Intel Pentium Gold G7400 3.7GHz/2C/4T/6M 46W D...
2               3      ✔  Intel Celeron G6900 3.4GHz/2C/2T/4M 46W DDR5 4800
3               4      ✔    Intel i9-12900 2.4GHz/16C/24T/30M 65W DDR5 4800
4               5      ✔    Intel i7-12700 2.1GHz/12C/20T/25M 65W DDR5 4800
...           ...    ...                                                ...
15183        6120      ✔                              Windows 10 Home 64 EM
15184        6120      ✔                              Windows 11 Home 64 EM
15185        6120      ✔                             F MB Dali FP5 3020E MP
15186        6121      ✔  USB Calliope Keyboard Black US English 103P  103P
15187        6122      ✔                              USB Fingerprint Mouse

[15188 rows x 3 columns]


In [165]:
import sqlite3 
from py2neo import Graph, Node, Relationship
g = Graph('neo4j://localhost:7687', auth = ('neo4j', 'ys1203303'))
g.run("MATCH (n) DETACH DELETE n") 
for i in range(len(components)):
        node1 = Node(
            "parent",
            Name = newrules[i],
            Component = components[i],
            originalRule = rules[i],
            Type = "Derive",
            ProjectName = project_name[i],
            Version = version[i],
            owner = owner[i], 
            date = date[i],
            ruleIndex = OD_rules[i])
        g.create(node1) 
        section_df = df[df['Section_id'] == i + 1] 
        print(section_df) 
        for s, ss in section_df.iterrows():
            node2 = Node(
                "son", 
                Name = ss['name'],
                Component = components[i],
                originalRule = rules[i],
                Type = "Derive",
                ProjectName = project_name[i],
                Version = version[i],
                owner = owner[i], 
                date = date[i],
                ruleIndex = OD_rules[i])
            g.create(node2)
            if ss['Status'] == '✔':
                relationship = Relationship(node2, "should", node1) 
            else:
                relationship = Relationship(node2, "should not", node1) 
            g.create(relationship) 

   Section_id Status                                           name
0           1      ✔  Intel i3-12100 3.3GHz/4C/8T/12M 60W DDR5 4800
   Section_id Status                                               name
1           2      ✔  Intel Pentium Gold G7400 3.7GHz/2C/4T/6M 46W D...
   Section_id Status                                               name
2           3      ✔  Intel Celeron G6900 3.4GHz/2C/2T/4M 46W DDR5 4800
   Section_id Status                                             name
3           4      ✔  Intel i9-12900 2.4GHz/16C/24T/30M 65W DDR5 4800
   Section_id Status                                             name
4           5      ✔  Intel i7-12700 2.1GHz/12C/20T/25M 65W DDR5 4800
   Section_id Status                                             name
5           6      ✔   Intel i5-12400 2.5GHz/6C/12T/18M 65W DDR5 4800
6           6      ✔  Intel i5-12400F 2.5GHz/6C/12T/18M 65W DDR5 4800
7           6      ✔   Intel i5-12500 3.0GHz/6C/12T/18M 65W DDR5 4800
   Section_id St